Introduction

Pour simplifier les unités de monnaie seront les euros.

In [1238]:
# Package pour les expressions régulières
import re

# Package pour la gestion des objets json
import json

# Package manipuler les fichiers et répertoires
import os

# Package pour les tableaux et dataframe
import numpy as np
import pandas as pd
from pandas.errors import MergeError

# Package pour analyse statistique
import scipy.stats as ss
from scipy.stats import loguniform

# Package pour la visualisation graphique
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns

# Typing des fonctions
from typing import List

# Packages scikit-learn pour modélisation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    OneHotEncoder, OrdinalEncoder, LabelEncoder,
    RobustScaler, StandardScaler
)
from scikitplot.metrics import plot_cumulative_gain
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    cross_val_predict, cross_val_score
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import VarianceThreshold, RFECV
from boruta import BorutaPy
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from  xgboost import XGBClassifier
from sklearn.calibration import (
    CalibratedClassifierCV,
    calibration_curve
)
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from shap import (
    TreeExplainer, summary_plot, 
    KernelExplainer,Explainer
)

from sklearn import set_config


In [1239]:
pd.options.display.max_columns = None #  pour afficher toutes les colonnes de la base
pd.options.display.max_rows = None  # pour afficher toutes les lignes de la base
pd.set_option('display.max_colwidth', None) # on ne définit pas de largeur max des colonnes afin d'éviter de tronquer leur contenu à l'affichage
set_config(display='diagram') # pour afficher la visualisation graphique des données ou des processus

In [1240]:
seed = 42

# 1. Importation et merge des données

# 5 Modélisation

Ici on va tester 4 grandes familles de modèle
* logistique régression
* SVC
* RandomForest
* XGBoost

In [1223]:
cat_ordinal_cols = []
cat_one_hot_encoder = []

for i,var in enumerate(cat_cols) : 
    if all(isinstance(var, (int, float)) for var in X_train[var]) : 
        cat_ordinal_cols.append(var)
    else :
        cat_one_hot_encoder.append(var)

## 5.1 Pipeline de transformation

In [1224]:
# initialisation des Imputers
num_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
cat_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Initialisation des encoders
ordinal_encoder = OrdinalEncoder(categories= [["No", "Yes"],["No", "Yes"],["No", "Yes"],["No", "Yes"],["No", "Yes"],["No", "Yes"]])  # SLOSLOà tester pour être sure
one_hot_encoder = OneHotEncoder()
# Standard scaler
scaler = StandardScaler()


#make pipeline
cat_ordinal_transformer = make_pipeline( cat_imputer, ordinal_encoder)
cat_one_hot_encoder_transformer = make_pipeline( cat_imputer, one_hot_encoder)
cat_other_transformer = make_pipeline( cat_imputer)
num_transformer = make_pipeline( num_imputer, scaler )
# Définition du préprocesseur
preprocessor = ColumnTransformer(
        transformers=[
            ("cat_ord", cat_ordinal_transformer, bool_cols),
             ("cat_oh", cat_one_hot_encoder_transformer, cat_one_hot_encoder),
             ("cat_", cat_other_transformer, cat_ordinal_cols),
             ("numerical", num_transformer, num_cols)            
        ]
)


## 5.2 Définition des modèles

In [1225]:
clf_models  = { "Logistique Régression" :  LogisticRegression(max_iter = 1000, random_state= seed), 
               "SVC Linéaire" : SVC(kernel='linear', random_state=seed),
              # "Nearest Neighbors" : KNeighborsClassifier(n_jobs= -1),
               "XGBboost" : XGBClassifier(random_state =42),
               "Random Forest" : RandomForestClassifier(random_state=42)
               }

In [1226]:
for model in clf_models : 
        # Définition du pipeline de transformation pipe_mod1_ord_enc
        pipe = Pipeline([
                ('preprocessor', preprocessor),
                ('regressor', clf_models[model])])
        #print(pipe)
        pipe.fit(X_train, y_train)
        #display(pipe)


        # Entrainement du modèle à l'aide de (X_train, y_train), et prédiction de y_train_pred
        roc_auc = cross_val_score(pipe,X_train, y_train,cv=3, n_jobs=-1, scoring ='roc_auc')
        print(f"Modele {model} : Score Val roc_auc : {roc_auc}")

ValueError: Selected columns, ['family_size_bins', 'number_of_referrals_bins', 'family_size_bins', 'number_of_referrals_bins', 'contract', 'offer', 'payment_method', 'internet_type', 'total_of_services_add', 'total_of_services_add', 'tenure_in_months_bins', 'tenure_in_months_bins'], are not unique in dataframe

## 5.3 Fine Tuning

### 5.3.1 Régression logistique

#### 5.3.1.1 avec GridSearch

Paramètres suivants sur lesqeul j'ai envie de jouer :
regarder en même temps le temps CPU

solver = {new-cholesky : L2, None
          liblinear  : {L1, L2}
          lbfgs : {l2, None}}

C = [0.1, 1, 10, 100]

tol = 1e-5, 1e-4, 1e-3

In [ ]:

pipe_lr = Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', LogisticRegression(max_iter = 1000, random_state= seed))]) #
    
grid = {"classifier__solver" : ["lbfgs", "liblinear"],
        "classifier__penalty" : ["l2"],
                "classifier__C" : [0.1, 1, 5 ,10, 100],
                "classifier__tol" : [0.0001, 0.001,0.01,0.1]}
        
grid_search = GridSearchCV(estimator = pipe_lr, param_grid = grid, cv = 5, n_jobs = 5, return_train_score = True, scoring = "roc_auc")
grid_search.fit(X_train, y_train)  
result_lr_gs = pd.DataFrame(grid_search.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la Regression Logistique avec Grid Search : \nles meilleurs params : {grid_search.best_params_}\nle meilleur score : {grid_search.best_score_}")
result_lr_gs[["params","mean_test_score","mean_train_score"]].head()
            

#### 5.3.1.2 avec RandomizedSearch

#### 5.3.1.3 Hyperparamètres retenus

Les parametre retenus ont dans le gridsearch 
* C*=0.2941
* PENALTY = L2
* SOLVER =LIBLINEAR
* TOL=0.02962



### 5.3.2 SVC

#### 5.3.2.1 avec GridSearch

In [ ]:



pipe_svc = Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', SVC(kernel='linear', random_state=seed, max_iter=1000))]) #
    
grid_svc = { "classifier__C" : [0.1, 1, 5 ,10, 100],
"classifier__class_weight" : ["balanced", "none"],
                "classifier__tol" : [0.0001, 0.001,0.01,0.1]}
        
grid_search_svc = GridSearchCV(estimator = pipe_svc, param_grid = grid_svc, cv = 5,n_jobs = 5, return_train_score = True, scoring = "roc_auc")
grid_search_svc.fit(X_train, y_train)   

result_svc_gs = pd.DataFrame(grid_search_svc.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la SVC avec Grid Search : \nles meilleurs params : {grid_search_svc.best_params_}\nle meilleur score : {grid_search_svc.best_score_}")
result_svc_gs[["params","mean_test_score","mean_train_score"]].head()


#### 5.3.2.2 avec RandomizedSearch

In [ ]:
grid_random = {"classifier__class_weight" : ["balanced", "none"],
                "classifier__C" : loguniform(0.01, 100),
                "classifier__tol" : loguniform(0.00001,0.11)}

        
grid_rs_svc = RandomizedSearchCV(estimator = pipe_svc, param_distributions= grid_random, cv = 5, n_jobs = 5,n_iter=200,return_train_score = True, scoring = "roc_auc")
grid_rs_svc.fit(X_train, y_train)   
result_svc_rs = pd.DataFrame(grid_rs_svc.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la SVC avec Random Search : \nles meilleurs params : {grid_rs_svc.best_params_}\nle meilleur score : {grid_rs_svc.best_score_}")
result_svc_rs[["params","mean_test_score","mean_train_score"]].head()


#### 5.3.2.3 Hyperparamètres retenus

Parametre retenu :
grid seach
{'classifier__C': 0.1, 'classifier__class_weight': 'balanced', 'classifier__tol': 0.0001}	0.882455	0.886631

### 5.3.3 Forêt aléatoire

#### 5.3.3.1 avec GridSearch

In [ ]:
pipe_rf = Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', RandomForestClassifier(random_state=42))]) #
    
grid = { "classifier__n_estimators" : [100, 200, 500, 1000],
"classifier__max_depth" : [3,4,5,6]}
        
grid_search_rf = GridSearchCV(estimator = pipe_rf, param_grid = grid, cv = 5,n_jobs = 5, return_train_score = True, scoring = "roc_auc")
grid_search_rf.fit(X_train, y_train)   
result_rf_gs = pd.DataFrame(grid_search_rf.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la rf avec grid search : \nles meilleurs params : {grid_search_rf.best_params_}\nle meilleur score : {grid_search_rf.best_score_}")
result_rf_gs[["params","mean_test_score","mean_train_score"]].head()

#### 5.3.3.2 avec RandomizedSearch

In [ ]:
grid_random = {"classifier__max_depth" : range(3,7,1),
                "classifier__n_estimators" : range(100,600,50)}

        
grid_rs_rf = RandomizedSearchCV(estimator = pipe_rf, param_distributions= grid_random, cv = 5, n_jobs = 5,n_iter=80,return_train_score = True, scoring = "roc_auc")
grid_rs_rf.fit(X_train, y_train)   
result_rf_rs = pd.DataFrame(grid_rs_rf.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la rf avec Random Search : \nles meilleurs params : {grid_rs_rf.best_params_}\nle meilleur score : {grid_rs_rf.best_score_}")
result_rf_rs[["params","mean_test_score","mean_train_score"]].head()


#### 5.3.3.3 Hyperparamètres retenus

{'classifier__max_depth': 6, 'classifier__n_estimators': 200}	0.886168	0.908774

{'classifier__n_estimators': 400, 'classifier__max_depth': 6}	0.886262	0.908853

### 5.3.4 Modèle XgBoost

#### 5.3.4.1 avec GridSearch

In [ ]:
pipe_xgb = Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', XGBClassifier( objective= 'binary:logistic',random_state =42))]) #
    
grid = { "classifier__n_estimators" : [100,200,500],
"classifier__max_depth" : [3,4,5,6],
                "classifier__eta" : [0.01,0.05,0.1],"classifier__lambda" : [0.1,1,10]}
        
grid_search_xgb = GridSearchCV(estimator = pipe_xgb, param_grid = grid, cv = 5,n_jobs = 5, return_train_score = True, scoring = "roc_auc")
grid_search_xgb.fit(X_train, y_train)   

result_gs_xgb = pd.DataFrame(grid_search_xgb.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour le xsb avec grid Search : \nles meilleurs params : {grid_search_xgb.best_params_}\nle meilleur score : {grid_search_xgb.best_score_}")
result_gs_xgb[["params","mean_test_score","mean_train_score"]].head()

    

#### 5.3.4.2 avec RandomizedSearch

In [ ]:
    
gri_random = {"classifier__n_estimators" : range(100,600,50),
                "classifier__max_depth" : range(3,7,1),
                "classifier__eta" : loguniform(0.01,0.3),
                 "classifier__lambda" : loguniform(0.1,10)}
        
grid_rs_xgb = RandomizedSearchCV(estimator = pipe_xgb, param_distributions= gri_random, cv = 5, n_jobs = 5, n_iter=144,return_train_score = True, scoring = "roc_auc")
grid_rs_xgb.fit(X_train, y_train)   

result_rs_xgb = pd.DataFrame(grid_rs_xgb.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour le xgb avec rs Search : \nles meilleurs params : {grid_rs_xgb.best_params_}\nle meilleur score : {grid_rs_xgb.best_score_}")
result_rs_xgb[["params","mean_test_score","mean_train_score"]].head()

#### 5.3.4.3 Hyperparamètres retenus

gs :
{'classifier__eta': 0.01, 'classifier__lambda': 1, 'classifier__max_depth': 4, 'classifier__n_estimators': 500}	0.898699	0.923246


rs 
{'classifier__eta': 0.03962445310841409, 'classifier__lambda': 4.5245010993028485, 'classifier__max_depth': 4, 'classifier__n_estimators': 150}	0.898761	0.922564

## 5.4 Evaluation

In [ ]:
#@title Chargez la fonction de calcul du score de Spiegelhalter
def score_spiegelhalter(y_true, y_pred):
    numerateur = np.sum(np.multiply(y_true - y_pred, 1 - 2 * y_pred))
    denominateur = np.sqrt(
        np.sum(
            np.multiply(
                np.multiply(np.power(1 - 2 * y_pred, 2), y_pred), 1 - y_pred
            )
        )
    )

    return numerateur / denominateur

In [ ]:
clf_models  = { "Logistique Régression" :  LogisticRegression(max_iter = 1000, random_state= seed,C=2941, penalty="l2", solver = 'liblinear', tol = 0.02962  ), 
               "SVC Linéaire" : SVC(kernel='linear', probability=True,random_state=seed, C = 0.1, class_weight="balanced", tol = 0.0001),
               "Random Forest" : RandomForestClassifier(random_state=42, n_estimators=400, max_depth=6),
               "XGBboost" : XGBClassifier(random_state =42,reg_lambda = 1, eta = 0.01, max_depth = 4, n_estimators = 500)
               }



In [ ]:
for model in clf_models : 
        # Définition du pipeline de transformation pipe_mod1_ord_enc
        pipe = Pipeline([
                ('preprocessor', preprocessor),
                ('regressor', clf_models[model])])
        #print(pipe)
        pipe.fit(X_train, y_train)
        y_train_proba = pipe.predict_proba(X_train )
        y_test_proba = pipe.predict_proba(X_test)
        auc_train = roc_auc_score(y_train, y_train_proba[:,1])
        auc_test = roc_auc_score(y_test, y_test_proba[:,1])
        ## enregister ici les résultt

        # # Entrainement du modèle à l'aide de (X_train, y_train), et prédiction de y_train_pred
        print(f"Modele {model} : Train  : {auc_train}, Test : {auc_test}")

        print(f"SPH avant calibration: {score_spiegelhalter(y_test, y_test_proba[:,1])}")

Les 4 modeles présentent de très bons résultats, ils sont robustes tous les 4 on peut utiliser le XGBboost par la suite.

### 5.4.1 Courbe de Lift

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,4))
plot_cumulative_gain(y_test, y_test_proba, title='Courbe de Lift', ax=ax1,  title_fontsize='13', text_fontsize='10')

# x10 = 0.1
# ypos = y_test.index(x10)
# y10 = y_test_proba[ypos]

# #Labeling the graph (ymax+1 is defining the distance from the word to the point)   
# ax1.annotate('l0.1', xy=(x10, y10), xytext=(x10,y10 ))



ax1.set_xlabel(str("Moyenne des probabilités"))
ax1.set_ylabel(f'y_pred > 0 _ nbins = {j}')
ax1.grid(visible=True, which='major', axis='y')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(True)
ax1.spines['left'].set_visible(False)
del ax1.lines[0]                 # delete the desired class plot
#ax1.lines[1].set_color("red")
ax1.lines[0].set_linewidth(1),ax1.lines[1].set_linewidth(1)
ax1.lines[0].set_color("blue")
#ax1.set_xticks(range(0,1,0.1))
ax1.legend().set_visible(False)  # hide the legend
ax1.legend().get_texts()[0].set_text("Classe 1 coucou")  # turn the legend back on
plt.show()

Figure ci-dessus : changer les set-ticks, ajouter une ligne verticale des 5%, 10%, 15%, 20, 25%
voir si stop la courbe et la refaire ....

### 5.4.2 Calibration

#### 5.4.2.1 Séparation du jeu de test en calibrated et test

In [ ]:
X_cal, X_test_cal, y_cal, y_test_cal = train_test_split(X_test, y_test, test_size= 0.5, random_state=seed)

In [ ]:
y_test_cal_proba_before_calibration = pipe.predict_proba(X_test_cal)

In [ ]:
SPH = score_spiegelhalter(y_test_cal, y_test_cal_proba_before_calibration[:,1])
SPH

#### 5.4.2.1 Courbe de Calibration - avant calibration

In [ ]:
#@title Chargez la fonction de calcul de la courbe de calibration via sklearn
def sklearn_calibration(y_true, y_pred, n_bins=20):

    prob_true, prob_pred = calibration_curve(
        y_true, y_pred,
        n_bins=n_bins,
        strategy="quantile"
    )

    df = pd.DataFrame({"prob_pred":prob_pred, "prob_true":prob_true})

    return df

In [ ]:

# Définition de la palette et de la légende pour le type de carburant
#palette = {"GAZOIL" : "orange", "ESSENCE" : "green", "AUTRE" : "grey"}
#handle_ = [plt.Line2D([], [], color=col, marker='o', linestyle='', markersize=8, alpha=0.5) for col in palette.values()]
#legend_  = [carburant for carburant in palette.keys()]

#0. Création de la figure
fig, axe = plt.subplots(figsize=(5,5), constrained_layout=True)


# Itération sur les 6 variables
j = 20#for i, j in enumerate(range(5,20,5)) :
sklearn_calibration_df = sklearn_calibration(y_test, y_test_proba[:,1], 20)#1. Création du scatterplot
sns.scatterplot(ax=axe, data=sklearn_calibration_df, x = 'prob_pred', y="prob_true")#,palette = palette, legend = False)
# sur le scatterplot coé = f(conso_mixte) on rajoute les 3 droites suivantes :
sns.lineplot(ax=axe,  x = [0, 1], y=[0,1],alpha=0.5)#, color=palette["ESSENCE"])

#2. Gestion des axis et labels
axe.set_xlabel(str("Moyenne des probabilités"))
axe.set_ylabel(f'y_pred > 0 _ nbins = {j}')
# if j == 0 :
# axe[i,j].set_ylabel('Emission CO2 (g/km)')

#3. Ajout des lignes horizontales
axe.grid(visible=True, which='major', axis='y')
axe.spines['top'].set_visible(False)
axe.spines['right'].set_visible(False)
axe.spines['bottom'].set_visible(True)
axe.spines['left'].set_visible(False)
  
#3. Ajout de la légende
#0handles, labels = plt.gca().get_legend_handles_labels()
#plt.legend(handle_, legend_, title = "Type de carburant", loc='center left', bbox_to_anchor= (1.04, 1), borderaxespad=0, frameon=False, fontsize = 10)

#4. Ajout du titre principal
#fig.suptitle(f'Emissions de CO2 (g/km) en fonction de la puissance maximale, de la masse, de leur ratio ainsi que des consommations en carburant en milieu urbain, extra urbain et mixte.')

fig.show()

* Le modele est mal calibré |SH| > 2
* Technique de calibration sur les sets : Puisque notre classifieur a déjà été entrainé, on met de coté une partie de jeu de données de test pour la calibration

#### 5.4.2.2 Resultat après calibration

In [ ]:
# Instanciation du modèle de calibration

calibration_clf = CalibratedClassifierCV(base_estimator=pipe, method='isotonic', cv="prefit", n_jobs=-1)
calibration_clf.fit(X_cal,y_cal)
y_test_cal_proba = calibration_clf.predict_proba(X_test_cal)

In [ ]:
SPH = score_spiegelhalter(y_test_cal, y_test_cal_proba[:,1])
SPH

In [ ]:

# Définition de la palette et de la légende pour le type de carburant
#palette = {"GAZOIL" : "orange", "ESSENCE" : "green", "AUTRE" : "grey"}
#handle_ = [plt.Line2D([], [], color=col, marker='o', linestyle='', markersize=8, alpha=0.5) for col in palette.values()]
#legend_  = [carburant for carburant in palette.keys()]

#0. Création de la figure
fig, axe = plt.subplots( figsize=(5,5), constrained_layout=True)


# Itération sur les 6 variables
sns.scatterplot(ax=axe, data=sklearn_calibration_df, x = 'prob_pred', y="prob_true")#,palette = palette, legend = False)

#sklearn_before_calibration_df = sklearn_calibration(y_test_cal, y_test_cal_proba_before_calibration[:,1], 20)#1. Création du scatterplot
sns.scatterplot(ax=axe, data=sklearn_calibration_df, x = 'prob_pred', y="prob_true",color = "red")#,palette = palette, legend = False)
sklearn_calibration_af_df = sklearn_calibration(y_test_cal, y_test_cal_proba[:,1], 20)#1. Création du scatterplot
sns.scatterplot(ax=axe, data=sklearn_calibration_af_df, x = 'prob_pred', y="prob_true",color = "blue")#,palette = palette, legend = False)



# sur le scatterplot coé = f(conso_mixte) on rajoute les 3 droites suivantes :
sns.lineplot(ax=axe,  x = [0, 1], y=[0,1],alpha=0.5)#, color=palette["ESSENCE"])
    
#2. Gestion des axis et labels
axe.set_xlabel(str("Moyenne des probabilités"))
axe.set_ylabel(f'y_pred > 0 _ nbins = {j}')
# if j == 0 :
#     axe[i,j].set_ylabel('Emission CO2 (g/km)')

#3. Ajout des lignes horizontales
axe.grid(visible=True, which='major', axis='y')
axe.spines['top'].set_visible(False)
axe.spines['right'].set_visible(False)
axe.spines['bottom'].set_visible(True)
axe.spines['left'].set_visible(False)
  
#3. Ajout de la légende
#0handles, labels = plt.gca().get_legend_handles_labels()
#plt.legend(handle_, legend_, title = "Type de carburant", loc='center left', bbox_to_anchor= (1.04, 1), borderaxespad=0, frameon=False, fontsize = 10)

#4. Ajout du titre principal
#fig.suptitle(f'Emissions de CO2 (g/km) en fonction de la puissance maximale, de la masse, de leur ratio ainsi que des consommations en carburant en milieu urbain, extra urbain et mixte.')

fig.show()

## 5.5 Explicatibilité

### 5.5.1 Analyse globale selon la permutation feature importance

In [ ]:
X_test_prepro = pd.DataFrame(pipe.named_steps.preprocessor.transform(X_test), columns=tot_cols)

In [ ]:
print(f"################# FEATURE IMPORTANCE###################")
model_fi = permutation_importance(model_, X_test_prepro, y_test, scoring="roc_auc", n_repeats= 30, random_state=seed)
feature_importance= pd.DataFrame({"names" : tot_cols, "importance" : np.round(model_fi['importances_mean'],4),
                                          "importance_sstd" : np.round(model_fi['importances_std'],4), })
print(feature_importance.sort_values(by="importance",ascending=False))


### 5.5.2 Analyse global selon SHAP

In [ ]:
X_test_prepro = pd.DataFrame(pipe.named_steps.preprocessor.transform(X_test), columns=tot_cols)

In [ ]:
shap_explainer = TreeExplainer(model = pipe.named_steps.regressor)
shap_values = shap_explainer(X_test_prepro)
shap.summary_plot(shap_values = shap_values, features=X_test_prepro, plot_size=(10,6))

réfléchir à comment présenter les choses et faire un Feature_importance correct
Analyser le problème avec la calibration.


In [ ]:

grid_random = {"classifier__solver" : ["lbfgs", "liblinear"],
                "classifier__penalty" : ["l2"],
                "classifier__C" : loguniform(0.1, 100),
                "classifier__tol" : loguniform(0.0001,0.3)}
        
grid_randomized = RandomizedSearchCV(estimator = pipe_lr, param_distributions= grid_random, cv = 5, n_jobs = 5, n_iter=200,return_train_score = True, scoring = "roc_auc")
grid_randomized.fit(X_train, y_train)   

result_lr_rs = pd.DataFrame(grid_randomized.cv_results_).sort_values("mean_test_score", ascending = False)
print(f"Pour la Regression Logistique avec Grid Search : \nles meilleurs params : {grid_randomized.best_params_}\nle meilleur score : {grid_randomized.best_score_}")
result_lr_rs[["params","mean_test_score","mean_train_score"]].head()
            